Manipulators are verbs

In [1]:
class Color(str: String = "")

class Pixel(x: Int, y: Int) {
  def paint(color: Color): Unit = {
    // Implementation here
  }
}

val center = new Pixel(50, 50)
center.paint(new Color("red"))

defined class Color
defined class Pixel
center: Pixel = ammonite.$sess.cmd1$Helper$Pixel@34998709

When an object allows us to manipulate, the name has to be a verb, and there must be no return value.

Examples of refactoring


In [1]:
class Document {
  def write(content: java.io.InputStream): Unit = {
    // Implementation here
  }
}

defined class Document

The example above shows a Document class with a write method that serves as a manipulator. Following the principle, it has a verb name and returns Unit (void).

However, what if we need to know how many bytes were saved? Let's refactor according to the principles:


In [2]:
// Problem: This violates our principle - a manipulator returning a value
class DocumentProblem {
  def write(content: java.io.InputStream): Int = {
    // Implementation that writes and returns bytes written
    42 // Dummy return for example
  }
}

// Solution: Separate the concerns
class Document {
  def output(): OutputPipe = {
    new OutputPipe()
  }
}

class OutputPipe {
  def write(content: java.io.InputStream): Unit = {
    // Implementation here
  }

  def bytes: Int = {
    // Return number of bytes written
    42 // Dummy return for example
  }

  def time: Long = {
    // Return time taken for writing
    123L // Dummy return for example
  }
}

// Usage example
val doc = new Document()
val pipe = doc.output()
pipe.write(new java.io.ByteArrayInputStream("Hello".getBytes))
val bytesWritten = pipe.bytes
val timeElapsed = pipe.time

defined class DocumentProblem
defined class Document
defined class OutputPipe
doc: Document = ammonite.$sess.cmd2$Helper$Document@69ee3a5e
pipe: OutputPipe = ammonite.$sess.cmd2$Helper$OutputPipe@6d2ee5ef
bytesWritten: Int = 42
timeElapsed: Long = 123L

Notice how we've refactored to maintain the principle:
1. `output()` is a builder - it returns a new object and has a noun name
2. `write()` is a manipulator - it has a verb name and returns Unit
3. `bytes` and `time` are builders - they have noun names and return values


The Builder Pattern and its concerns


In [4]:
// Builder Pattern example (though the author recommends against it)
class Book (
  val author: String = "",
  val title: String = "",
  val pages: List[String] = List.empty
) {
  def withAuthor(author: String): Book = {
    new Book(author, title, pages)
  }

  def withTitle(title: String): Book = {
    new Book(author, title, pages)
  }

  def withPage(page: String): Book = {
    new Book(author, title, pages :+ page)
  }
}

// Usage
val myBook = new Book()
  .withAuthor("John Doe")
  .withTitle("Clean Code")
  .withPage("Chapter 1")

defined class Book
myBook: Book = ammonite.$sess.cmd4$Helper$Book@18ef9967

The Builder Pattern methods follow the principle - they have noun-like names (with a prefix) and return values. However, as noted in the text, the author generally advises against this pattern as it often leads to less cohesive, larger objects.


A better approach to complex objects with many properties


In [ ]:
// Instead of a large Book class with a builder, break it into smaller objects
class Author(val name: String)
class Title(val text: String)
class Page(val content: String)

class Book(val author: Author, val title: Title, val pages: List[Page]) {
  def printInfo(): Unit = {
    println(s"${title.text} by ${author.name}, ${pages.size} pages")
  }
}

// Usage
val book = new Book(
  new Author("Jane Smith"),
  new Title("Elegant Objects"),
  List(new Page("Introduction"), new Page("Chapter 1"))
)
book.printInfo()

This approach creates smaller, more focused objects rather than using the Builder Pattern for complex initialization.


## Why This Principle Matters

This strict separation between builders and manipulators makes code:

1. **More readable** - Method names clearly indicate what they do
2. **More predictable** - Builders don't change state, manipulators don't return values
3. **More maintainable** - Single responsibility principle applied at the method level
4. **More testable** - Easier to test methods with clear, focused responsibilities


## Handling Complex Cases

Sometimes we need both information and manipulation. The solution is to create specific objects for each responsibility:


In [5]:
// Example: Managing a database connection
class Database(url: String) {
  // Builder - returns a connection object
  def connection: Connection = {
    new Connection(url)
  }

  // Builder - returns statistics object
  def stats: Statistics = {
    new Statistics(url)
  }
}

class Connection(url: String) {
  // Manipulator - executes a query
  def execute(sql: String): Unit = {
    println(s"Executing $sql on $url")
  }

  // Manipulator - closes the connection
  def close(): Unit = {
    println("Connection closed")
  }
}

class Statistics(url: String) {
  // Builder - returns query count
  def queryCount: Int = {
    // Implementation to get query count
    42
  }

  // Builder - returns connection time
  def connectionTime: Long = {
    // Implementation to get connection time
    1000L
  }
}

// Usage
val db = new Database("jdbc:postgresql://localhost:5432/mydb")
val conn = db.connection
conn.execute("SELECT * FROM users")
conn.close()

val stats = db.stats
println(s"Executed ${stats.queryCount} queries")
println(s"Connection time: ${stats.connectionTime}ms")


Executing SELECT * FROM users on jdbc:postgresql://localhost:5432/mydb
Connection closed
Executed 42 queries
Connection time: 1000ms


defined class Database
defined class Connection
defined class Statistics
db: Database = ammonite.$sess.cmd5$Helper$Database@7e2ce878
conn: Connection = ammonite.$sess.cmd5$Helper$Connection@194543a4
stats: Statistics = ammonite.$sess.cmd5$Helper$Statistics@6d254bb0

### Boolean Results

Boolean methods are an exception to the naming rules. They are builders (they return values), but instead of using nouns, they should use adjectives:


In [6]:
class File(path: String) {
  // Good: Using adjectives for Boolean methods
  def empty: Boolean = {
    // Implementation to check if file is empty
    true
  }

  def readable: Boolean = {
    // Implementation to check if file is readable
    true
  }

  def present: Boolean = {
    // Better than "exists" - "is present" sounds right
    true
  }

  // Bad examples (would be better as adjectives)
  def isEmpty: Boolean = empty // Not recommended - redundant "is" prefix
  def exists: Boolean = present // Not recommended - use "present" instead
}

case class User(name: String, age: Int) {
  // Good: "equal to" sounds right
  def equalTo(other: User): Boolean = {
    this.name == other.name && this.age == other.age
  }

  // Good: adjective for Boolean result
  def adult: Boolean = {
    age >= 18
  }
}

// Usage examples showing natural reading in conditionals
val file = new File("/tmp/data.txt")
val user = new User("John", 25)
val otherUser = new User("Jane", 30)

if (file.empty) {
  println("The file is empty") // Reads naturally: "if file is empty"
}

if (file.readable) {
  println("The file can be read") // Reads naturally: "if file is readable"
}

if (user.adult) {
  println("User is an adult") // Reads naturally: "if user is adult"
}

if (user.equalTo(otherUser)) {
  println("Users are the same") // Reads naturally: "if user is equal to other user"
}


The file is empty
The file can be read
User is an adult


defined class File
defined class User
file: File = ammonite.$sess.cmd6$Helper$File@7ef5dca2
user: User = User(name = "John", age = 25)
otherUser: User = User(name = "Jane", age = 30)

## Summary of Boolean Method Naming Rules

1. **Boolean methods are builders** (they return values)
2. **Use adjectives for Boolean methods**, not nouns or verbs
3. **Don't use the "is" prefix** in the method name
4. **Test readability** by mentally placing "is" before the name - it should sound correct
5. **Examples of good names**:
   - `empty()` instead of `isEmpty()`
   - `readable()` instead of `canRead()`
   - `negative()` instead of `isNegative()`
   - `equalTo(obj)` instead of `equals(obj)`
   - `present()` instead of `exists()`

This naming convention makes conditionals read more naturally, as the Boolean methods essentially answer the question "is this object [adjective]?"

## 2.5 Don't use public constants

Public constants create global coupling and reduce cohesion.
Prefer keeping values private to the class that uses them, or pass them in as small objects (dependencies) instead of sharing a global constant.

In [11]:
// Bad example: global constant-like object (avoid)
object ConstantsBad {
  val CrLf: String = "\r\n"
}

class RowsBad2(private val all: List[String]) {
  def print(p: java.io.PrintStream): Unit = {
    all.foreach { row => p.print(s"$row${ConstantsBad.CrLf}") }
  }
}

defined object ConstantsBad
defined class RowsBad2

Better alternatives:

- Keep the value private and local to the class.
- Or encapsulate the value as an object and inject it.

In [12]:
class Rows2(private val all: List[String]) {
  private val CrLf: String = "\r\n"
  def print(p: java.io.PrintStream): Unit =
    all.foreach { row => p.print(s"$row$CrLf") }
}

trait LineSeparator { def value: String }
class CrLf extends LineSeparator { val value = "\r\n" }
class Lf extends LineSeparator { val value = "\n" }

class Records2(private val all: List[String], private val sep: LineSeparator) {
  def write(out: java.io.Writer): Unit = {
    all.foreach { rec =>
      out.write(rec)
      out.write(sep.value)
    }
  }
}

defined class Rows2
defined trait LineSeparator
defined class CrLf
defined class Lf
defined class Records2

## 2.5.1 Introduction of coupling

Public constants introduce hidden, hard-coded dependencies. When multiple classes reach out to a shared global value, they become tightly coupled to it and to each other, making behavior changes unpredictable.

Consider these two classes that both rely on a shared line separator constant:


In [13]:
// Bad: shared global constant couples unrelated classes to the same value
object ConstantsGlobal {
  val CrLf: String = "\r\n"
}

class RecordsCoupled(private val all: List[String]) {
  def write(out: java.io.Writer): Unit = {
    all.foreach { rec =>
      out.write(rec)
      out.write(ConstantsGlobal.CrLf) // hard dependency here
    }
  }
}

class RowsCoupled(private val all: List[String]) {
  def print(p: java.io.PrintStream): Unit = {
    all.foreach { row =>
      p.print(s"$row${ConstantsGlobal.CrLf}") // hard dependency here
    }
  }
}

defined object ConstantsGlobal
defined class RecordsCoupled
defined class RowsCoupled

Now, RecordsCoupled.write, RowsCoupled.print, and ConstantsGlobal.CrLf are all coupled. If we change CrLf (for example, to a platform-specific or protocol-specific value), the behavior of both classes changes, and it’s unclear whether those changes are acceptable. One user may be printing to the console, while another may be composing an HTTP message where the line ending is mandated and must not change.

The constant sits in a global scope without semantic context. We can’t tell how it’s used or which usages are safe to alter. This erodes maintainability: many objects depend on a value whose meaning varies by context.

A better design is to inject small objects that carry the context (as shown above with LineSeparator and Records2/Rows2), so each class depends on what it needs, and changes are localized.


## 2.5.2 Loss of cohesion

Public constants push unrelated semantics into consumers. Records and Rows shouldn’t know how to apply a line ending; that’s a separate responsibility. Instead of sharing a naked value, share behavior with a tiny object that owns the meaning of the value.

Key ideas:
- Don’t share data, share behavior (micro classes).
- Couple through contracts, not through globals.
- Semantics live with the object that knows them.


In [23]:
// A tiny object that owns the semantics of “append CRLF to a line”
final case class CRLFString(private val origin: String) {
  override def toString: String = s"$origin\r\n"
}

// Usage in code that writes records (no constant, no formatting logic here)
final class Records3(private val all: List[String]) {
  def write(out: java.io.Writer): Unit =
    all.foreach { rec =>
      out.write(CRLFString(rec))
    }
}


defined class CRLFString
defined class Records3

In [25]:
// Usage in code that prints rows (PrintStream can print any object via toString)
final class Rows3(private val all: List[String]) {
  def print(p: java.io.PrintStream): Unit =
    all.foreach { row =>
      p.print(CRLFString(row))
    }
}

private val rows = new Rows3(List("row1", "row2"))
rows.print(System.out) // prints with CRLF


row1
row2


defined class Rows3

Platform-aware behavior stays encapsulated in the same micro class. The contract doesn’t change; only behavior does.


In [29]:
final class SafeCRLFString(private val origin: String) {
  override def toString: String = {
    val os = java.lang.System.getProperty("os.name", "").toLowerCase
    if (os.contains("win"))
      throw new IllegalStateException("We're on Windows, can't use CRLF, sorry")
    s"$origin\r\n"
  }
}

object SafeCRLFString {
  def apply(origin: String): SafeCRLFString = new SafeCRLFString(origin)
}


defined class SafeCRLFString
defined object SafeCRLFString

Another example: replace HTTP method constants with tiny classes that configure a request.


In [ ]:
// A very small HTTP request stub; the method is part of its state
trait Request { def fetch(): String }

final class HttpRequest extends Request {
  private var method: String = "GET"
  def withMethod(m: String): HttpRequest = { method = m; this }
  override def fetch(): String = s"Fetched with method=$method"
}

// Instead of HttpMethods.POST constant, use a tiny configurator object
final class PostRequest(private val origin: HttpRequest) extends Request {
  override def fetch(): String = origin.withMethod("POST").fetch()
}

// Usage
val bodyViaPost: String = new PostRequest(new HttpRequest).fetch()

## 2.6 Be immutable

Make classes immutable to keep them small, cohesive, decoupled, and easy to reason about. An immutable object doesn’t change its state after creation; operations return new objects instead of mutating existing ones.

Key ideas:
- Prefer vals over vars; keep fields private and final by default.
- Methods that “modify” return new instances (no setters).
- Small, focused objects compose behavior instead of sharing mutable state.


### Mutable vs. Immutable example (money arithmetic)


In [ ]:
// Bad: mutable object with in-place modification
final class MutableMoney(private var dollars: Int) {
  def mul(factor: Int): Unit = { dollars *= factor }
  override def toString: String = s"$$$dollars"
}

val fiveMut = new MutableMoney(5)
fiveMut.mul(10)
println(fiveMut) // "$50" — original object changed


In [32]:
// Good: immutable object that returns a new instance
final case class ImmutableMoney(private val dollars: Int) {
  def mul(factor: Int): ImmutableMoney = copy(dollars = dollars * factor)
  override def toString: String = s"$$$dollars"
}

val five = ImmutableMoney(5)
val fifty = five.mul(10)
println(five)   // "$5"
println(fifty)  // "$50"


$5
$50


defined class ImmutableMoney
five: ImmutableMoney = ImmutableMoney(dollars = 5)
fifty: ImmutableMoney = ImmutableMoney(dollars = 50)

Why this is better:
- No hidden state changes; values read like they are named.
- Easier to test and reason about; no ordering surprises.
- Safer to share across threads (no synchronization needed for reads).


### Practical rules of thumb for immutability in Scala

- Use case classes and vals; avoid public vars and setters.
- Prefer tiny value objects over exposing raw primitives.
- Keep constructors doing all necessary initialization; don’t leave objects half-baked.
- If you need a different value, construct a different object.


### Lazy loading without mutability

Lazy loading is often cited as a reason to mutate. In Scala, lazy val gives you safe, one-time initialization without exposing mutability.


In [35]:
// A source for HTML content
trait HtmlSource { def fetch(uri: String): String }

// Immutable page with lazy-loaded content
final class PageLazy(private val uri: String, private val src: HtmlSource) {
  // Thread-safe, computed once on first access, then cached
  lazy private val html: String = src.fetch(uri)
  def content: String = html
}

final class StubSource extends HtmlSource {
  override def fetch(uri: String): String = s"<html><body>Loaded $uri</body></html>"
}

val page = new PageLazy("https://example.org", new StubSource)
println(page.content) // triggers fetch once
println(page.content) // returns cached result


<html><body>Loaded https://example.org</body></html>
<html><body>Loaded https://example.org</body></html>


defined trait HtmlSource
defined class PageLazy
defined class StubSource
page: PageLazy = ammonite.$sess.cmd35$Helper$PageLazy@30ab8e82

Notes:
- lazy val in Scala is thread-safe and evaluates at most once per instance.
- Keep dependencies small (like HtmlSource) and inject them; PageLazy stays cohesive and testable.


### A tiny, reusable memoization helper

For computed values that aren’t fields, wrap the computation in a tiny object and memoize it.


In [36]:
final class Memoized[A](thunk: => A) {
  // Evaluate on first access and cache thereafter
  lazy val value: A = thunk
}

// Usage: any expensive computation
def heavy(n: Int): Int = { println(s"computing for $n"); n * n }

val squared = new Memoized(heavy(21))
println(squared.value) // computes
println(squared.value) // cached


computing for 21
441
441


defined class Memoized
defined function heavy
squared: Memoized[Int] = ammonite.$sess.cmd36$Helper$Memoized@23e43f74

### Takeaways

- Don’t mutate objects; return new ones.
- Use lazy val for one-time, cached initialization without sacrificing immutability.
- Prefer micro-objects and explicit dependencies over shared mutable state.

### 2.6.1 Identity mutability

Immutable objects don’t suffer from “identity mutability.” The bug appears when an object’s equality and hash code depend on mutable state: after the object is used as a key in a map (or element of a set), mutating it breaks the collection’s invariants. Lookups become unpredictable and duplicates (by equality) may appear.


In [37]:
import scala.collection.mutable

// Bad: key equality depends on mutable state
final class MutableKeyMoney(private var _dollars: Int) {
  def dollars: Int = _dollars
  def mul(factor: Int): Unit = { _dollars *= factor }

  // Equality and hashCode depend on (mutable) dollars
  override def equals(obj: Any): Boolean = obj match {
    case that: MutableKeyMoney => this._dollars == that._dollars
    case _ => false
  }
  override def hashCode(): Int = java.lang.Integer.hashCode(_dollars)
  override def toString: String = s"$$$dollars"
}

val map = mutable.HashMap[MutableKeyMoney, String]()
val fiveK = new MutableKeyMoney(5)
val tenK  = new MutableKeyMoney(10)
map.put(fiveK, "five")
map.put(tenK,  "ten")

// Mutate one key after it has been placed in the map
fiveK.mul(2) // now equals(tenK) and hashCode matches tenK

println(map)                  // keys may look equal ($10, $10), but both entries still exist internally
println(map.get(fiveK))       // lookup may fail or be inconsistent (hash bucket mismatch)
println(map.get(tenK))        // may succeed, highlighting the broken invariant
println(map.keys.count(k => k == fiveK)) // > 1 by equality, revealing duplicates by equals()


HashMap($10 -> five, $10 -> ten)
Some(ten)
Some(ten)
2


import scala.collection.mutable

// Bad: key equality depends on mutable state

defined class MutableKeyMoney
map: HashMap[MutableKeyMoney, String] = HashMap($10 -> "five", $10 -> "ten")
fiveK: MutableKeyMoney = $10
tenK: MutableKeyMoney = $10
res37_5: Option[String] = None
res37_6: Option[String] = None

Why this happens:
- Hash-based collections place keys into buckets using hashCode computed at insertion time.
- If hashCode/equals depend on mutable fields and those fields change after insertion, the stored bucket no longer matches the key’s identity.
- Future lookups use the new hash/equality and search the wrong bucket; duplicates-by-equality can also appear.


Safe alternative: immutable keys (stable equality)


In [38]:
import scala.collection.mutable

// Good: immutable key with stable equals/hashCode
final case class MoneyKey(dollars: Int) {
  override def toString: String = s"$$$dollars"
}

val safe = mutable.HashMap[MoneyKey, String]()
safe.put(MoneyKey(5),  "five")
safe.put(MoneyKey(10), "ten")

// We “change” by creating a new key; the old entry remains valid
val fiveKey = MoneyKey(5)
safe.put(fiveKey.copy(dollars = fiveKey.dollars * 2), "ten-again")

println(safe)                 // Map($5 -> five, $10 -> ten, $10 -> ten-again)
println(safe.get(MoneyKey(10))) // deterministic and consistent


HashMap($5 -> five, $10 -> ten-again)
Some(ten-again)


import scala.collection.mutable

// Good: immutable key with stable equals/hashCode

defined class MoneyKey
safe: HashMap[MoneyKey, String] = HashMap(
  MoneyKey(dollars = 5) -> "five",
  MoneyKey(dollars = 10) -> "ten-again"
)
res38_3: Option[String] = None
res38_4: Option[String] = None
fiveKey: MoneyKey = MoneyKey(dollars = 5)
res38_6: Option[String] = Some(value = "ten")

Guidelines to avoid identity mutability bugs:
- Never use mutable objects as keys in HashMap/HashSet (or any hashed structure).
- Ensure equals/hashCode rely only on immutable state. Prefer case classes for value objects.
- Keep classes final with private vals; expose behavior, not setters.
- “Modifications” should produce new objects, leaving existing map keys valid forever.
